In [10]:
import csv
import random
import sys
from pathlib import Path

# ---- CONFIG ----
infile = r"C:\Users\aayus\Downloads\archive (2)\MSR_data_cleaned.csv"
outfile = r"C:\Users\aayus\Downloads\msr_reservoir_sample_5000.csv"
sample_size = 5000
# ----------------

# Try to set csv.field_size_limit to the largest supported value (robust)
max_int = sys.maxsize
while True:
    try:
        csv.field_size_limit(max_int)
        break
    except OverflowError:
        # reduce by factor of 10 until it fits
        max_int = max_int // 10
        if max_int <= 0:
            raise RuntimeError("Could not set csv.field_size_limit to a usable value on this platform")

print("Using csv.field_size_limit =", csv.field_size_limit())

# Reservoir sampling
with open(infile, mode='r', newline='', encoding='latin1', errors='replace') as fin:
    reader = csv.reader(fin)
    try:
        header = next(reader)
    except StopIteration:
        raise RuntimeError("Input file appears empty")

    reservoir = []
    for i, row in enumerate(reader, start=1):
        if i <= sample_size:
            reservoir.append(row)
        else:
            j = random.randrange(i)
            if j < sample_size:
                reservoir[j] = row

# Write out sample (using utf-8 to be safer for downstream tools)
Path(outfile).parent.mkdir(parents=True, exist_ok=True)
with open(outfile, mode='w', newline='', encoding='utf-8') as fout:
    writer = csv.writer(fout)
    writer.writerow(header)
    writer.writerows(reservoir)

print(f"Wrote {len(reservoir)} sampled rows to: {outfile}")


Using csv.field_size_limit = 9223372036854775807
Wrote 5000 sampled rows to: C:\Users\aayus\Downloads\msr_reservoir_sample_5000.csv


In [11]:
import pandas as pd
df = pd.read_csv(r"C:\Users\aayus\Downloads\msr_reservoir_sample_5000.csv")
df.to_parquet(r"C:\Users\aayus\Downloads\msr_reservoir_sample.parquet", index=False)